In [2]:
!pip install -q selenium

In [2]:
import pandas as pd
import csv
import time

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException

In [2]:
nyc_stores = pd.read_csv('data/nyc_boba_store50.csv')
stores = nyc_stores['alias'].tolist()
print(len(stores))

50
['bar-pa-tea-new-york-2', 'teado-tea-shop-new-york-6', 'tea-and-milk-astoria', 'yumpling-long-island-city', 'juice-vitality-new-york', 'hancos-brooklyn', 'tiger-sugar-new-york-9', 'taiyaki-nyc-williamsburg-brooklyn-7', 'yaya-tea-garden-new-york-3', 'surreal-creamery-new-york-2', 'vivi-bubble-tea-new-york-6', 'i-milky-沐白-new-york-4', 'sashimi-express-new-york-2', 'ten-rens-tea-time-new-york-2', 'mi-tea-new-york-4', 'patea-new-york-6', 'banh-mi-place-brooklyn', 'hanco-s-brooklyn', 'debutea-new-york', 'kung-fu-tea-new-york-31', 'cream-coffee-and-tea-shop-brooklyn-2', 'kung-fu-tea-new-york-35', 'bread-and-joe-brooklyn-2', 'yaya-tea-grand-new-york-2', 'smoove-ice-cream-new-york-2']
['kung-fu-tea-new-york-10', 'taiwan-bear-house-new-york', 'hawa-smoothies-and-bubble-tea-new-york', 'savory-thai-brooklyn-4', 'gong-cha-new-york-9', 'banhmigos-brooklyn-5', 'one-more-charm-brooklyn', 'bambu-new-york', 'biao-sugar-new-york', 'xing-fu-tang-new-york', 'dragon-tea-new-york-3', 'noodlecraft-long-is

In [7]:
driver = webdriver.Chrome('/Users/justinatran/Downloads/chromedriver')

/Users/justinatran/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  """Entry point for launching an IPython kernel.


In [4]:
def get_data(csvwriter, store_alias):
  reviews = driver.find_elements(By.CSS_SELECTOR, "p.comment__09f24__gu0rG span.raw__09f24__T4Ezm")
  username = driver.find_elements(By.CSS_SELECTOR, "span.fs-block a.css-1m051bw")
  user_loc = driver.find_elements(By.CSS_SELECTOR, "div.review__09f24__oHr9V span.css-qgunke") #first item in the list is 'Location'
  rating = driver.find_elements(By.CSS_SELECTOR, "div.review__09f24__oHr9V div.five-stars__09f24__mBKym")
  date = driver.find_elements(By.CSS_SELECTOR, "div.review__09f24__oHr9V span.css-chan6m")

  # create lists of all the items
  reviews_list = [r.text for r in reviews]
  username_list = [u.text for u in username]
  user_loc_list = [u.text for u in user_loc]
  rating_list = [r.get_attribute("aria-label") for r in rating]
  date_list = [r.text for r in date]

  for (date, username, loc, rating, review) in zip(date_list, username_list, user_loc_list, rating_list, reviews_list):
    csvwriter.writerow([store_alias, date, username, loc, rating, review])

In [5]:
def fetch_reviews(csv_writer, store_alias):
    while True:
        try:
            get_data(csv_writer, store_alias)
            # go next page
            driver.find_element(By.CSS_SELECTOR, "span.css-foyide a.next-link").click()
            WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "span.css-foyide a.next-link")))
            time.sleep(5)
            
        except (TimeoutException, WebDriverException) as e:
            print("Last page reached")
            break
    
    #driver.quit()
    


In [8]:
# get results and write to csv
with open('nyc_reviews.csv', 'w', encoding='utf-8') as csvfile:
  csv_writer = csv.writer(csvfile)
  # add column names as first row
  csv_writer.writerow(['store','date', 'username', 'user_loc', 'rating','reviews'])
  # get all data
  for store in stores:
    site = driver.get(f'https://www.yelp.com/biz/{store}')
    fetch_reviews(csv_writer, store)
  

Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached
Last page reached


## Checking Output Results

In [11]:
nyc_reviews_final = pd.read_csv('nyc_reviews.csv')
nyc_reviews_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4660 entries, 0 to 4659
Data columns (total 6 columns):
store       4660 non-null object
date        4660 non-null object
username    4660 non-null object
user_loc    4660 non-null object
rating      4660 non-null object
reviews     4660 non-null object
dtypes: object(6)
memory usage: 218.5+ KB


In [5]:
nyc_reviews_final.to_csv('nyc_reviews_final.csv', index=False)